# Importing the relevant libraries

In [ ]:
import pandas as pd

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.tree import DecisionTreeClassifier, plot_tree

from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

import matplotlib.pyplot as plt

import seaborn as sns

import numpy as np

# Data Preprocessing

### Importing the Database

In [ ]:
# Import the data from the CSV file
raw_data = pd.read_csv('ml_datasource.csv')

# Display the first 5 rows of the dataframe for preview
raw_data.head()

In [ ]:
data = raw_data.copy()

### Removing Outliers

In [ ]:
# Reset any modifications to the plotting context (sns) made via seaborn
sns.reset_orig()

# Set the font scale for seaborn plots
sns.set(font_scale=1.5)

# Initialize a grid of plots with specified dimensions
fig, axes = plt.subplots(3, 2, figsize=(20,20))

# Plotting distribution plots for each of the columns in the dataset
sns.kdeplot(data=data['days_on_platform'], ax=axes[0,0])
sns.kdeplot(data=data['minutes_watched'], ax=axes[0,1])
sns.kdeplot(data=data['courses_started'], ax=axes[1,0])
sns.kdeplot(data=data['practice_exams_started'], ax=axes[1,1])
sns.kdeplot(data=data['practice_exams_passed'], ax=axes[2,0])
sns.kdeplot(data=data['minutes_spent_on_exams'], ax=axes[2,1]);

plt.show()

In [ ]:
# Removing outliers based on 'minutes_watched', 'courses_started',
# and 'practice_exams_started', and 'minutes_spent_on_exams' fields
data_no_outliers = data[(data['minutes_watched'] <= 1000)
                            & (data['courses_started']<=10)
                            & (data['practice_exams_started']<=10)
                            & (data['minutes_spent_on_exams']<=40)]

In [ ]:
# Reset any modifications to the plotting context (sns) made via seaborn
sns.reset_orig()

# Set the font scale for seaborn plots
sns.set(font_scale=1.5)

# Initialize a grid of plots with specified dimensions
fig, axes = plt.subplots(3, 2, figsize=(20,20))

# Plotting distribution plots for each of the columns in the dataset
sns.kdeplot(data=data_no_outliers['days_on_platform'], ax=axes[0,0])
sns.kdeplot(data=data_no_outliers['minutes_watched'], ax=axes[0,1])
sns.kdeplot(data=data_no_outliers['courses_started'], ax=axes[1,0])
sns.kdeplot(data=data_no_outliers['practice_exams_started'], ax=axes[1,1])
sns.kdeplot(data=data_no_outliers['practice_exams_passed'], ax=axes[2,0])
sns.kdeplot(data=data_no_outliers['minutes_spent_on_exams'], ax=axes[2,1]);

plt.show()

### Checking for Multicollinearity

In [ ]:
# Printing the column names of the dataset (for reference)
data_no_outliers.columns.to_numpy()

In [ ]:
# Selecting the numerical columns for Variance Inflation Factor (VIF) calculation
variables = data_no_outliers[['days_on_platform',
                              'minutes_watched',
                              'courses_started', 
                              'practice_exams_started', 
                              'practice_exams_passed', 
                              'minutes_spent_on_exams']]

# Creating a DataFrame to store the VIF value for each feature
vif = pd.DataFrame()

# Computing the VIF for each selected feature using list comprehension
# Storing the values in a column called 'VIF'
vif['VIF'] = [variance_inflation_factor(variables.to_numpy(), i) for i in range(variables.shape[1])]

# Storing the names of the features in a column called 'features'
vif['features'] = variables.columns

# Displaying the DataFrame
vif

In [ ]:
# Dropping 'practice_exams' to prevent multicollinearity 
data_no_mult = data_no_outliers.drop('practice_exams_started', axis = 1)

# Displaying the first five rows of the new data
data_no_mult.head()

In [ ]:
# Selecting specific columns for new Variance Inflation Factor (VIF) calculation
variables = data_no_outliers[['days_on_platform',
                              'minutes_watched',
                              'courses_started', 
                              'practice_exams_passed', 
                              'minutes_spent_on_exams']]

# Computing the new VIF values for each selected feature
vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(variables.to_numpy(), i) for i in range(variables.shape[1])]
vif["features"] = variables.columns
vif

### Dealing with NaN Values

In [ ]:
# Checking the number of null values in each column
data_no_mult.isnull().sum()

In [ ]:
# Select rows from the 'data_no_mult' DataFrame where the 
# 'student_country' column has missing values (NaN).
data_no_mult.loc[ data_no_mult['student_country'].isna()]

In [ ]:
# Replacing NaN values with the string 'NAM'
data_no_nulls = data_no_mult.fillna('NAM', axis = 1)

In [ ]:
# Displaying records where 'student_country' is 'NAM'
data_no_nulls.loc[ data_no_nulls['student_country'] == 'NAM', 'student_country']

In [ ]:
# Re-checking the number of null values in each column after replacement
data_no_nulls.isnull().sum()

### Splitting the Data

In [ ]:
# Defining inputs (feature variables) and target (outcome variable)
inputs = data_no_nulls.drop(['purchased'],axis=1)
target = data_no_nulls['purchased']

In [ ]:
# Splitting the data into train and test sets, ensuring balanced classes with stratification
x_train, x_test, y_train, y_test = train_test_split(inputs, 
                                                    target, 
                                                    test_size=0.2, 
                                                    random_state=365,
                                                    stratify = target)

In [ ]:
# Display the first 5 rows of the DataFrame for preview
x_train.head()

### Encoding the Data

In [ ]:
# Initializing an ordinal encoder for categorical variables
enc = OrdinalEncoder(handle_unknown = 'use_encoded_value', 
                     unknown_value = 170);

In [ ]:
# Encoding the 'student_country' column in the training and testing datasets
# and storing the encoded variable in a new column called 'student_country_enc'
x_train['student_country_enc'] = enc.fit_transform(x_train['student_country'].to_numpy().reshape(-1, 1));
x_test['student_country_enc'] = enc.transform(x_test['student_country'].to_numpy().reshape(-1, 1));

# Dropping the original 'student_country' column after encoding
x_train = x_train.drop('student_country', axis = 1)
x_test = x_test.drop('student_country', axis = 1)

# Displaying the first five rows of the encoded training dataset
x_train.head()

In [ ]:
# We use the np.asarray() method to ensure data type consistency and compatibility.
# y_train values are converted to integers and x_train values are converted to floating-point numbers.

x_train_array = np.asarray(x_train, dtype = 'float')
y_train_array = np.asarray(y_train, dtype = 'int')

x_test_array = np.asarray(x_test, dtype = 'float')
y_test_array = np.asarray(y_test, dtype = 'int')

# Creating a Logistic Regression Model

In [ ]:
# Creating a logistic regression model using the sm.Logit function
log_reg = sm.Logit(y_train_array, x_train_array)

# Fitting the model to the training data
log_reg_results = log_reg.fit()

# Displaying a summary of the model results
log_reg_results.summary()

In [ ]:
# Generating predictions on the test set and rounding the predictions to nearest integer (0 or 1)
y_test_pred_log_reg = [round(log_reg_results.predict(x_test_array)[i], 0) 
                       for i in range(len(y_test_array))]

In [ ]:
# Resetting any modifications to the plotting context made via seaborn
sns.reset_orig()

# Displaying the confusion matrix from the model's predictions;
# 'magma' colormap is used for better visualization of the matrix;
ConfusionMatrixDisplay.from_predictions(
    y_test_array, y_test_pred_log_reg,
    cmap = 'magma'
);

plt.show()

# Creating a K-Nearest Neighbors Model

In [ ]:
# Define the parameter grid for the grid search
parameters_knn = {'n_neighbors':range(1, 51), 
                  'weights':['uniform', 'distance']}

In [ ]:
# Initialize a GridSearchCV object with KNeighborsClassifier estimator
# and accuracy as the scoring metric
grid_search_knn = GridSearchCV(estimator = KNeighborsClassifier(), 
                               param_grid = parameters_knn, 
                               scoring = 'accuracy')

In [ ]:
# Fit the grid search object on the training data
grid_search_knn.fit(x_train_array, y_train_array)

In [ ]:
# Display the best parameters and the corresponding score
grid_search_knn.best_params_, grid_search_knn.best_score_

In [ ]:
# Store the best estimator (model with optimal parameters) in knn_clf
knn_clf = grid_search_knn.best_estimator_

# Display the model
knn_clf

In [ ]:
# Predict the target variable for the test dataset using the optimal model
y_test_pred_knn = knn_clf.predict(x_test_array)

# Reset seaborn settings to default
sns.reset_orig()

# Display the confusion matrix using seaborn;
# Set the labels to the classes of the model;
# Choose the magma colormap for better visualization
ConfusionMatrixDisplay.from_predictions(
    y_test_array, y_test_pred_knn,
    labels = knn_clf.classes_,
    cmap = 'magma' 
);

plt.show()

In [ ]:
# Print the classification report, providing an overview of how well the classifier has performed
print(classification_report(y_test_array, 
                            y_test_pred_knn, 
                            target_names = ['0', '1']))

# Creating a Support Vector Machines Model

In [ ]:
# Create an instance of the MinMaxScaler. This will normalize the features to a range between -1 and 1
scaling = MinMaxScaler(feature_range=(-1,1))

# Fit the scaler to the training data and transform it
x_train_array_svc = scaling.fit_transform(x_train_array)

# Use the same scaler to transform the test data
x_test_array_svc = scaling.transform(x_test_array)

In [ ]:
# Set up a dictionary of parameters for grid search
parameters_svc = {'kernel':['linear', 'poly', 'rbf'], 
                  'C':range(1, 11),
                  'gamma': ['scale', 'auto']}

In [ ]:
# Initialize a GridSearchCV object with the SVC estimator and accuracy as the scoring metric
grid_search_svc = GridSearchCV(estimator = SVC(), 
                               param_grid = parameters_svc, 
                               scoring = 'accuracy')

In [ ]:
# Note: Fitting process can take some time due to the number of combinations
grid_search_svc.fit(x_train_array_svc, y_train_array)

In [ ]:
# Display the best model obtained from the grid search
grid_search_svc.best_estimator_

In [ ]:
# Store the best estimator (model with optimal parameters) in svc_clf
svc_clf = grid_search_svc.best_estimator_

In [ ]:
# Use the best estimator to predict the target variable for the test dataset
y_test_pred_svc = svc_clf.predict(x_test_array_svc)

# Reset seaborn settings to default
sns.reset_orig()

# Display the confusion matrix using seaborn;
# Use 'magma' colormap for better visualization
ConfusionMatrixDisplay.from_predictions(
    y_test_array, y_test_pred_svc,
    labels = svc_clf.classes_,
    cmap = 'magma'
);

plt.show()

In [ ]:
# Print the classification report to get precision, recall, f1-score, and support for each class
print(classification_report(y_test_array, 
                            y_test_pred_svc, 
                            target_names = ['0', '1']))

# Creating a Decision Trees Model

In [ ]:
# Define the parameters for the GridSearch. 
# Here we're trying different values for the ccp_alpha parameter
parameters_dt = {'ccp_alpha':[0, 
                              0.001, 
                              0.002, 
                              0.003, 
                              0.004, 
                              0.005]}

In [ ]:
# Initialize GridSearchCV with the estimator being a DecisionTreeClassifier with a fixed random state.
# The scoring metric used here is accuracy
grid_search_dt = GridSearchCV(estimator = DecisionTreeClassifier(random_state = 365), 
                              param_grid = parameters_dt, 
                              scoring = 'accuracy')

In [ ]:
# Fit a GridSearchCV object on the training data
grid_search_dt.fit(x_train_array, y_train_array)

In [ ]:
# Display the best estimator after grid search
grid_search_dt.best_estimator_

In [ ]:
# Assign the model with the best parameters to dt_clf
dt_clf = grid_search_dt.best_estimator_

In [ ]:
# Set the figure size for the plot
plt.figure(figsize=(15,10))

# Plot the decision tree. Feature names and class names are added for better interpretability
plot_tree(dt_clf, 
          filled=True, 
          feature_names = ['Days on platform', 
                           'Minutes watched', 
                           'Courses started',
                           'Practice exams passed', 
                           'Time spent on exams', 
                           'Student country encoded'], 
          class_names = ['Will not purchase', 
                         'Will purchase'])

# Display the plot
plt.show()

In [ ]:
# Use the best decision tree model to predict the target variable for the test dataset
y_test_pred_dt = dt_clf.predict(x_test_array)

In [ ]:
# Reset seaborn settings to default
sns.reset_orig()

# Display the confusion matrix using seaborn.
# Use 'magma' colormap for better visualization
ConfusionMatrixDisplay.from_predictions(
    y_test_array, y_test_pred_dt,
    labels = dt_clf.classes_,
    cmap = 'magma'
);

plt.show()

In [ ]:
# Print the classification report to get precision, recall, f1-score, and support for each class
print(classification_report(y_test_array, y_test_pred_dt))

# Creating a Random Forests Model

In [ ]:
# Initialize the RandomForestClassifier with ccp_alpha set to 0.0001
rf_clf = RandomForestClassifier(ccp_alpha = 0.0001, random_state = 365)

In [ ]:
# Train the random forest model on the training data
rf_clf.fit(x_train_array, y_train_array)

In [ ]:
# Use the trained random forest model to predict the target variable for the test dataset
y_test_pred_rf = rf_clf.predict(x_test_array)

In [ ]:
# Reset seaborn settings to default
sns.reset_orig()

# Display the confusion matrix using seaborn
# Use 'magma' colormap for better visualization
ConfusionMatrixDisplay.from_predictions(
    y_test_array, y_test_pred_rf,
    labels = rf_clf.classes_,
    cmap = 'magma' 
);

plt.show()

In [ ]:
# Print the classification report to get precision, recall, f1-score, and support for each class
print(classification_report(y_test_array, y_test_pred_rf))